### Automatización creación playlist Spotify con la extracción de tracks en formato CVS:

In [ ]:
# 1.Importaciones:

import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth # este 2ºAuth es para la autenticación de usuaria y poder meter la playlist en la cuenta
import os # librería de python para interactuar con el sistema operativo del pc y poder ocultar las credenciales guardándolas en las variables de entorno del sistema(esto prepara la librería para ese paso)
from pathlib import Path # es una biblio de Python para gestionar las rutas de archivos sin que que se esguince al usarlo en sistemas operativos distintos

In [ ]:
# esta celda es solo de control, para ver que la ruta al csv de las canciones, es la correcta

import os
# La r (raw string)es una medida de seguridad para evitar errores con los caracteres de escape (\) y que no os desesperéis tanto como yo para que os reconozca la ruta(si usáis windows incluidla si o si)
NOMBRE_ARCHIVO = r"C:\Users\Andrea\Desktop\ADALAB\da-promo-60-modulo2-equipo3\csv last.fm\country_2018_2022_canciones.csv" 

if os.path.exists(NOMBRE_ARCHIVO):
    print("El archivo SÍ existe. El problema está en la lectura de Pandas.")
else:
    print("El archivo NO existe. ¡Revisa el nombre del archivo o la ruta!")


In [ ]:
# 2.Credenciales y configuración :
# Ésto es porque en nuestras extracciones tenemos el id de las canciones, no el uri(que es cómo spotify indexa las canciones en las listas)


CLIENT_ID = "......................."  # los que genera la API
CLIENT_SECRET = "......................." 
USERNAME = "......................." # Ésto es el código alfanumérico de usuario de tu perfil (ojo, tiene que ser el del perfil desde el que generaste las credenciales de la API)

# aquí NOMBRE_ARCHIVO_STR es una cadena de texto(es funcional usando el mismo nombre para la variable, puras buenas prácticas de analitiquitaniñabuena)
NOMBRE_ARCHIVO_STR = r"C:\Users\Andrea\Desktop\ADALAB\da-promo-60-modulo2-equipo3\csv last.fm\country_2018_2022_canciones.csv"  
COLUMNA_ID = 'id' # como se llama la columna en nuestra extracción de tracks


# Carga y preparación de los uris de marras

try:
    # aquí NOMBRE_ARCHIVO_STR lo convertimos de verdad de la buena,en un objeto de ruta(path), porque la celda de control no vas a volver a ejecutarla una vez constates que encuentra tu archivo
    NOMBRE_ARCHIVO = Path(NOMBRE_ARCHIVO_STR)
    
    # Carganding el DataFrame
    # codificación UTF-8 (para reconocer ñ, acentos, etc.)
    df_final = pd.read_csv(NOMBRE_ARCHIVO, encoding='utf-8')
        
    # verificación y renombre uris 
    if COLUMNA_ID not in df_final.columns:
         raise KeyError(f"La columna '{COLUMNA_ID}' no se encontró en el archivo.")
    
    # hay que "construir" el uri para que Spotify nos haga caso 
    df_final = df_final.rename(columns={COLUMNA_ID: 'track_id'})
    df_final['uri_spotify'] = 'spotify:track:' + df_final['track_id'].astype(str)
    
    # una vez creados, mete los uris en la lista 
    uris_a_agregar = df_final['uri_spotify'].dropna().tolist()

    print(f" Archivo '{NOMBRE_ARCHIVO_STR}' cargado y columna '{COLUMNA_ID}' check!")
    print(f"Total de canciones listas para añadir: {len(uris_a_agregar)}.")

except FileNotFoundError:
    print(" ¡HORROR-ERROR!: El archivo sigue sin encontrarse")
    print("Revisa que esté todo bien escrito, amiga")
    
except KeyError as e:
    print(f" ERROR en las columnas, ojo!: {e}")
    uris_a_agregar = []
except Exception as e: # si es cualquier otro error random que python tiene de regalo para ti, distinto de los anteriores...
    print(f" ERROR: El archivo pudo cargarse, pero hubo un problema de codificación. Intenta cambiar el encoding. Ojo: {e}")
    uris_a_agregar = []
    
# Si hay error, la lista queda vacía
if 'uris_a_agregar' not in locals():
    uris_a_agregar = []


In [ ]:
# 3.Autenticación de tu perfil de usuaria en Spotify

REDIRECT_URI = #'https://developer.spotify.com/' Entra de nuevo en tu perfil en la API, en el dashboard y en la app que creaste para las extracciones, puede ponerte otra cosa distinta que a mi
SCOPE = 'playlist-modify-public' 

print("--- Iniciando el flujo de autenticación de usuaria... ---")

try:
    # configuración de la autenticación 
    sp_oauth = spotipy.SpotifyOAuth(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        scope=SCOPE,
        username=USERNAME,
        show_dialog=True # Fuerza la ventana de inicio de sesión. 
    )

    # obtener el Token de Acceso
    # Te va a abrir una pestaña en el navegador para que des permiso a que se ejecute todo dentro de la propia aplica de Spotify
    
    token_info = sp_oauth.get_access_token(as_dict=False, check_cache=True)
    
    # conexión con el token 
    sp_user = spotipy.Spotify(auth=token_info)
    
    print("✅ Autenticación de usuaria completada. ¡Podemos crear la playlist!")

except Exception as e:
    print(f"ENÉSIMO ERROR en la autenticación. Revisa el ID/Secret/USERNAME y la REDIRECT URI, pero no desesperes aún! ¡Casi lo tienes, titi!")
    print(f"Detalle del error: {e}")
    sp_user = None


In [ ]:
# 4. Creación de la playlist y volcado de las canciones:

# verificamos que la conexión de usuaria (sp_user) y la lista de URIs no estén vacías
if 'sp_user' in locals() and sp_user is not None and uris_a_agregar:
    
    # Puse de nombre de la playlist el del proyecto + el nombre de la extracción de canciones en el archivo, por identificarla y darle sentido a las 5 playlist juntas(cambiad vuestro género)
    playlist_name = "Proyecto MusicStreamStupendo - Country Hits 2018-2022" 
    
    print("\n--- Ya estamos creando y añadiendo TEMAZOS a tu Playlist... ---")

    try:
        # crea la playlist vacía
        new_playlist = sp_user.user_playlist_create(
            user=USERNAME, 
            name=playlist_name, 
            public=True,
            description="Las canciones más representativas de nuestro análisis de datos de Country (2018-2022)." # podéis escribir lo que queráis aquí, en realidad
        )
        playlist_id = new_playlist['id']
        
        print(f"🎉 Playlist '{playlist_name}' creada exitosamente, ¡hacker, que eres una hacker!.")
        print(f"ID de la Playlist: {playlist_id}")

        # añade las canciones en bloques (chunks) de 100, porque es el tope de uris que permiten agregar de cada vez, si pones una cifra más alta(o no pones nada), peta, OJO!
        chunk_size = 100
        for i in range(0, len(uris_a_agregar), chunk_size):
            chunk = uris_a_agregar[i:i + chunk_size]
            sp_user.playlist_add_items(playlist_id, chunk)

        print(f"✅ ¡{len(uris_a_agregar)} canciones añadidas con todo el éxito del mundo, querida!")
        print(f"Busca la playlist '{playlist_name}' en tu app de Spotify.")
        
    except Exception as e:
        print(f" Ooootro problemita al crear o rellenar la playlist.")
        print(f"Detalle del error: {e}")

elif sp_user is None:
    print("⚠️ Sorry, no se pudo crear la playlist porque la autenticación falló en el paso anterior. Revísalo, anda)")
elif not uris_a_agregar:
    print("⚠️ No se pudo crear la playlist porque la lista de canciones a agregar está vacía. Chequea ese paso, que ya lo tienes, veña!")